In [1]:
import string

class TOKEN:
    def __init__(self, classe, lexema, tipo):
        self.classe = classe
        # Transformando o parâmetro lexema obtido como lista para uma string
        self.lexema = ''.join(lexema)
        self.tipo = tipo
    
    # Define o que aparece quando se utiliza print(token)
    def __str__(self):
        return f'Classe={self.classe}, lexema={self.lexema}, Tipo={self.tipo}'

'''
Nova exception herdando de Exception para diferenciar quando utiliza try... except
'''
# 
class AlphabetError(Exception):
    pass

'''
Tabela inicializada com palavras reservadas e ids encontrados no programa fonte

'''
TABELA_DE_SIMBOLOS = [
    TOKEN('inicio', 'inicio', 'inicio'),
    TOKEN('varinicio', 'varinicio', 'varinicio'),
    TOKEN('varfim', 'varfim', 'varfim'),
    TOKEN('escreva', 'escreva', 'escreva'),
    TOKEN('leia', 'leia', 'leia'),
    TOKEN('se', 'se', 'se'),
    TOKEN('entao', 'entao', 'entao'),
    TOKEN('fimse', 'fimse', 'fimse'),
    TOKEN('fim', 'fim', 'fim'),
    TOKEN('inteiro', 'inteiro', 'inteiro'),
    TOKEN('literal', 'literal', 'literal'),
    TOKEN('real', 'real', 'real')
]

'''
Dicionário de dicionários representando a tabela de transição do autômato
Para mudar de estado utiliza-se estado = tabela_de_transicao[estado atual][chave] na qual as chaves são os símbolos no
estado atual para mudar de estado
'''
tabela_de_transicao = {
    0: {
        '(': 1,
        ')': 2,
        ';': 3,
        ',': 4,
        'EOF': 5,
        '+': 6,
        '-': 6,
        '*': 6,
        '/': 6,
        'letra': 7,
        '"': 8,
        '{': 10,
        '<': 12,
        '>': 14,
        '=': 15,
        'digito': 16
    },
    1: {},
    2: {},
    3: {},
    4: {},
    5: {},
    6: {},
    7: {
        'letra': 7,
        'digito': 7,
        '_': 7
    },
    8: {
        'curinga': 8,
        '"': 9
    },
    9: {},
    10: {
        'curinga': 10,
        '}': 11
    },
    11: {},
    12: {
        '=': 15,
        '>': 15,
        '-': 13
    },
    13: {},
    14: {
        '=': 15
    },
    15: {},
    16: {
        'digito': 16,
        '.': 17,
        'E': 19,
        'e': 19
    },
    17: {
        'digito': 18
    },
    18: {
        'digito': 18,
        'E': 19,
        'e': 19
    },
    19: {
        'digito': 21,
        '+': 20,
        '-': 20
    },
    20: {
        'digito': 21
    },
    21: {
        'digito': 21
    }
}

'''
Quando o autômato termina uma cadeia em algum desses estados sabemos qual tipo de token estamos lidando
'''
estados_finais = {
    0: 'estado inicial',
    1: 'AB_P',
    2: 'FC_P',
    3: 'PT_V',
    4: 'VIR',
    5: 'EOF',
    6: 'OPA',
    7: 'id',
    9: 'Lit',
    11: 'Comentário',
    12: 'OPR',
    13: 'ATR',
    14: 'OPR',
    15: 'OPR',
    16: 'Num',
    18: 'Num',
    21: 'Num'
}

'''
Algumas listas úteis
'''
letras = list(string.ascii_letters)
digitos = list(string.digits)
delimitadores = [' ', '\n', '\t']
alfabeto = list(letras + digitos + delimitadores + [
    ',', ';', ':', '.', '!', '?', '\\', '*', '+', '-', '/', '(', ')', '{', '}',
    '[', ']', '<', '>', '=', "'", '"'
])

In [2]:
'''
Essa função recebe o estado e o lexema para classificar e retornar o novo token
'''
def retorna_TOKEN(estado, lexema):
    # Verifica a cadeia completa por string sem fim ou caractere fora do alfabeto
#     for l in lexema:
#         if ((estado == 8 or estado == 10) and l == '\n') or (l not in alfabeto):
#             return TOKEN('ERROR', lexema, None)

    if estado == 8 or estado == 10:
        return TOKEN('ERROR', lexema, None)
#     for l in lexema:
#         if ((estado == 8 or estado == 10) and l == '\n') or (l not in alfabeto):
#             return TOKEN('ERROR', lexema, None)
        
    # Se estado == 1 então é um id ou uma palavra reservada
    if estado == 7:
        # Se já estiver na tabela de símbolos então o token da tabela de símbolos é retornado
        for token in TABELA_DE_SIMBOLOS:
            # Como lexema é uma lista é necessário usar join para comparar com uma string
            if token.lexema == ''.join(lexema):
                return token
        # Caso não tenha retornado então significa que foi encontrado um id que não está na tabela de símbolos
        novo_id = TOKEN('id', lexema, None)
        TABELA_DE_SIMBOLOS.append(novo_id)
        return novo_id
    
    # Definindo o tipo do token de acordo com a descrição do trabalho
    elif estado == 16:
        tipo = 'inteiro'
        
    elif estado == 18 or estado == 21:
        tipo = 'real'
        
    elif estado == 9:
        tipo = 'literal'
        
    else:
        tipo = None
    
    # Saída padrão caso não seja um id ou palavra reservada
    return TOKEN(estados_finais[estado], lexema, tipo)

'''
Essa função cria a chave para a transição de estados na tabela de transições de acordo com o estado atual e o caractere
recebido.

Utiliza-se as chaves "letra", "digito" e "curinga" ao invés do próprio caractere para não confundir com outros caracteres
iguais e para evitar a criação de um estado para cada letra e dígito.
'''
def chave(caractere, estado):
        if (estado == 16 or estado == 18) and (caractere =='e' or caractere == 'E'):
            chave = 'e'
        elif (estado == 8 and caractere != '"') or (estado == 10 and caractere != '}'):
            chave = 'curinga'
        elif caractere in letras:
            chave = 'letra'
        elif caractere in digitos:
            chave = 'digito'
        else:
            chave = caractere
            
        return chave


'''
Analisador léxico.

Recebe o código fonte como uma string e o lê caractere a caractere, unindo-os em lexemas que serão reconhecidos como
pertencentes a um padrão.
'''    
def SCANNER(fonte, posicao, linha, coluna):
    # Estado inicial
    estado = 0
    # Lexema é uma string, mas está sendo tratada como lista para facilitar operações com string
    lexema = []
    
    # Lê todos os caracteres do fonte
    while posicao <= len(fonte):
        try:
            # Se não pertence ao alfabeto gera uma exceção
            if fonte[posicao] not in alfabeto:
                raise AlphabetError
            # Caso pertença ao alfabeto realiza a transição de estados
            estado = tabela_de_transicao[estado][chave(fonte[posicao], estado)]
            
        # Foi encontrado um caractere que não pertence ao alfabeto
        except AlphabetError:
            # Caso exista um lexema sendo lido quando esse caractere foi encontrado retorna o token desse lexema primeiro
            if (len(lexema) > 0) and (estado in estados_finais):
                return retorna_TOKEN(estado, lexema), posicao, linha, coluna
            # Caso contrário mostra onde o caractere foi encontrado e continua análise
            print(f'ERRO LÉXICO - Caractere inválido na linguagem: {fonte[posicao]}. Linha {linha}, coluna {coluna}')
            lexema.append(fonte[posicao])
        
        # Quebra de padrão: Foi feita uma tentativa de fazer uma transição inexistente naquele estado
        except KeyError:
            # Caso haja um lexema e está em um estado final, então é porque já chegou ao fim
            if (len(lexema)) > 0 and (estado in estados_finais.keys()):
                return retorna_TOKEN(estado, lexema), posicao, linha, coluna
            # Exponenciação incompleta
            elif estado not in estados_finais.keys() and estado == 19:
                print(f'ERRO LÉXICO - Exponenciação incompleta. Linha {linha}, coluna {coluna}')                       
                return TOKEN('ERROR', lexema, None), posicao, linha, coluna
        # Chegou ao fim da string fonte
        except IndexError:
            # Há um lexema que deve ser retornado antes do EOF
            if len(lexema) > 0:
                return retorna_TOKEN(estado, lexema), posicao, linha, coluna
            # Sai do loop e retorna o EOF
            else: 
                break
        
        # Ocorreu a transição normalmente
        else:
            # Se o delimitador for \n então atualiza linha e coluna
            if fonte[posicao] == '\n':
                linha = linha + 1
                coluna = 0
                if estado == 8:
                    print(f'ERRO LÉXICO - Literal incompleto. Linha {linha}, coluna {coluna}')
                    return retorna_TOKEN(estado, lexema), posicao, linha, coluna
                elif estado == 10:
                    print(f'ERRO LÉXICO - Comentário incompleto. Linha {linha}, coluna {coluna}')
                    return retorna_TOKEN(estado, lexema), posicao, linha, coluna
            # Junta caractere ao lexema
            # Não ignora delimitadores dentro de comentários e literais
            if (fonte[posicao] not in delimitadores) or (estado == 8 or estado == 10):
                lexema.append(fonte[posicao])
        # Atualiza contadores
        finally:
            posicao = posicao + 1
            coluna = coluna + 1
            
    # Fim do arquivo fonte
    return TOKEN('EOF', 'EOF', None), posicao, linha, coluna

In [3]:
'''
Função main().

Lê o arquivo e faz um loop enquanto chama o SCANNER().
'''
def main():
    with open('..\\Teste\\teste3.txt') as file:
        fonte = file.read()
        posicao = 0
        linha = 1
        coluna = 1
        
        while posicao <= len(fonte):
            token, posicao, linha, coluna = SCANNER(fonte, posicao, linha, coluna)
            print(token)

In [4]:
main()

Classe=inicio, lexema=inicio, Tipo=inicio
Classe=varinicio, lexema=varinicio, Tipo=varinicio
Classe=literal, lexema=literal, Tipo=literal
Classe=id, lexema=A, Tipo=None
Classe=PT_V, lexema=;, Tipo=None
Classe=inteiro, lexema=inteiro, Tipo=inteiro
Classe=id, lexema=B, Tipo=None
Classe=VIR, lexema=,, Tipo=None
Classe=id, lexema=D, Tipo=None
Classe=VIR, lexema=,, Tipo=None
Classe=id, lexema=E, Tipo=None
Classe=PT_V, lexema=;, Tipo=None
Classe=real, lexema=real, Tipo=real
Classe=id, lexema=C, Tipo=None
Classe=PT_V, lexema=;, Tipo=None
Classe=varfim, lexema=varfim, Tipo=varfim
Classe=PT_V, lexema=;, Tipo=None
Classe=escreva, lexema=escreva, Tipo=escreva
Classe=Lit, lexema="Digite B:", Tipo=literal
Classe=PT_V, lexema=;, Tipo=None
Classe=leia, lexema=leia, Tipo=leia
Classe=id, lexema=B, Tipo=None
Classe=PT_V, lexema=;, Tipo=None
Classe=escreva, lexema=escreva, Tipo=escreva
Classe=Lit, lexema="Digite A:", Tipo=literal
Classe=PT_V, lexema=;, Tipo=None
Classe=leia, lexema=leia, Tipo=leia
Classe